# Scraping Twitter Handles (UK)

This notebook consists the data scraping part of the project, namely, scraping UK Twitter handles from Twitter using the Twitter API.

In [7]:
# Import dependancies
import tweepy
import re
import requests
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pickle
import time
import itertools

def userids_to_usernames(userids):
    client = tweepy.Client(
        bearer_token="AAAAAAAAAAAAAAAAAAAAAJhtiwEAAAAAnSH7LB8LCFaBffd9NrgflhozHQ8%3Dgl3augKz2Chu0iqikKNcszMsnvQxzPURyiCyQx0DjNwhDDpddO",
        consumer_key="x0ZhcTVGaRmabZ0wzw7cA6TFt",
        consumer_secret="CuI7c9JT8d8PkqkjK73JmTmSiowSQVLpQSAD7KlPwX2DfvKsm7",
        access_token="1454490866597441542-qkkzuAWHrfd84M2DDSBiHQtxjGi6GV",
        access_token_secret="n9X2PFuzNlxfgJdb54Uqkj0SzLIxQDWqW50ZYFZvfdPBi",
    )
    
    response = client.get_users(ids=userids)
    user_names = [user.username for user in response.data]
    errors = [error['value'] for error in response.errors]
    return user_names, errors

In [8]:
# Load data
data_df = pd.read_csv("files/cleaned_twitter_data_UK.csv",index_col=0, dtype = {'user_id': str})

/Users/huayuan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
# Inspect first five elements
data_df.head(5)

,tweet_ID,user_id,keywords,tweet_timestamp,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,emotion,tweet,len_content
1,1234290271774543872,248819855,covid,2020-03-02 01:31:47,0.380,0.611,0.412,0.218,0.460,-1.0,F,HEALTH EMERGENCY / INTERNATIONAL COVID-19 CORO...,8.0
2,1234291397202776065,248819855,covid,2020-03-02 01:36:15,0.408,0.505,0.422,0.218,0.417,-1.0,F,HEALTH EMERGENCY / INTERNATIONAL COVID-19 CORO...,23.0
3,1234292673260879872,447663369,corona,2020-03-02 01:41:20,0.448,0.566,0.419,0.267,0.415,-1.0,F,Corruption scaring corona virus from Africa &g...,7.0
5,1234294077564375040,1903820431,corona,2020-03-02 01:46:54,0.490,0.529,0.447,0.426,0.475,0.0,NaN,Starting up with a cold and lightly concerned ...,41.0
6,1234294853330182144,20973388,covid,2020-03-02 01:49:59,0.406,0.613,0.458,0.271,0.460,-1.0,F,"The UK government is considering all options, ...",27.0


In [10]:
data_df = data_df[data_df['user_id'].notna()]

# Inspect shape
data_df.shape

(593237, 13)

In [11]:
# Get list of tweet IDs to scrape
user_IDs = data_df["user_id"].tolist()

user_IDs = list(set(user_IDs))

# Take the first 100 tweets (for testing)
user_IDs_test = user_IDs[:100]
user_array = np.asarray(user_IDs_test)

# Partition into 10 equally sized arrays
split_array = np.array_split(user_array, 10)

for i in range(len(split_array)):
    split_array[i] = list(split_array[i])

In [12]:
# Inspect length
print(len(np.asarray(user_IDs)))

256898


In [14]:
# Get array with tweet IDs
user_array_final = np.asarray(user_IDs)

# Split into equally sized partitions
split_array_final = np.array_split(user_array_final, 2569)

for i in tqdm(range(len(split_array_final))):
    split_array_final[i] = list(split_array_final[i])

# Save list of tweet IDs to pickle file
with open('pickles/list_user_ids_UK.pkl', 'wb') as f:
    pickle.dump(split_array_final, f)

# Inspect list
print(len(split_array_final))

# Inspect final element
print(len(split_array_final[-1]))

100%|██████████| 2569/2569 [00:00<00:00, 27945.85it/s]


2569
99


In [15]:
print(split_array_final)

[['1191354362649600000', '393939390', '283150478', '179892930', '1060480019590451200', '242669344', '16211562', '19436360', '1236587393240702976', '47645992', '107449281', '162819697', '873410442009477120', '113349586', '2609473670', '1231658917119262721', '162100226', '471085181', '2394010110', '39085694', '1051061366', '58819263', '2172435486', '45937278', '520668002', '37639430', '4841551383', '375841804', '102603185', '1515952453', '226557715', '1241377362777985026', '1253997409006432256', '743191951', '2851663839', '3545943016', '381710376', '1228738187452919816', '258020976', '795764338770530304', '767915949299073024', '2957906388', '1252213342158102530', '744336558233427968', '491562636', '1226949954549186560', '944327853155475456', '129182760', '811325478359461892', '420688410', '91073393', '450798351', '3913455557', '1884129218', '338040863', '726800844029579264', '150661180', '297494239', '87709645', '4339501396', '1175798791', '19312177', '61864216', '1181307831552741378', '

In [16]:
%%time
tweet_list = []
error_list = []

## Retrieve tweets from tweet IDs ##
for i in tqdm(range(len(split_array_final))):
    try:
        tweets, errors = userids_to_usernames(split_array_final[i])   
    except:
        time.sleep(900) # Twitter API only allows for max 900 calls per 15 min. (900 secs) timeframe; sleep and continue
        tweets, errors = userids_to_usernames(split_array_final[i])  

    tweet_list.append(tweets)
    error_list.append(errors)

# Save tweets
with open('pickles/users_UK.pkl', 'wb') as f:
    pickle.dump(tweet_list, f)

# Save non-existent tweets
with open('pickles/errors_users_UK.pkl', 'wb') as f:
    pickle.dump(error_list, f)

100%|██████████| 2569/2569 [2:17:04<00:00,  3.20s/it]     

CPU times: user 1min 18s, sys: 8.38 s, total: 1min 27s
Wall time: 2h 17min 4s


In [17]:
# Open tweets list pickle for inspection
with open('pickles/users_UK.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

# Inspect length
print(len(mynewlist))

2569


In [18]:
# Inspect first tweet
print(mynewlist[0])

['wanserver', 'GillianEBest', '_SonOfBen_', 'EntelechyInc', 'evo_recordsuk', '_CoolCare', 'cgoomansingh', 'BBCSajidIqbal', 'King72Pp', 'catrionaacarter', 'Amshire', '__Tinsley__', 'JFieldPhD', 'EmeraldBuiltEnv', 'SGMF_SeaChange', 'GarethSapstead', 'thaysen13', 'vbroszek', 'diannamelroseUK', 'HevLou76', 'carlhickey24', 'AlexDold', 'ArcticSabrina', 'gaf01', 'daddyanddumbell', 'Vhero', 'njoreillydebre2', 'petestephenson2', 'ranjbains', 'CoralDeVille', 'petrameeson', 'une_orchidee', 'Si73119968', 'keftyron', 'rtgrffn', 'AliceLsAndDs', 'oliviamayafield', 'myonlinegifts', 'laneboyr6', 'WSussexBrexit', 'ZelloTac', 'sadboiyams', 'FuzzyDunlop_1', 'TrueBrit45', 'ColleenMJT', 'kasfriedrice', 'northernapphoto', 'amyinthedesert', 'Jones_2552', 'Chris_E_Dunn', 'KMCMadhyamaka', '1867_Owl', 'ludlowmortimer', 'DeluxeAccounts', 'ThisLeadenPall', 'TomDuggers', 'ChildOfAlba', 'greggcopemillar', 'WalesAudit', 'PositiveAction', 'Donnawatson66', 'ojobson', 'lizvmeer', 'kerrymlove', 'BenjiRobertson', 'emma_ey

In [19]:
# Check overall number of successfully scraped tweets
final_list = list(itertools.chain.from_iterable(mynewlist))
print(len(final_list))

256268


In [20]:
# Check final tweet
print(final_list[-1])

nictuff
